# Preliminary functions

In [19]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
# use Stratified KFold instead to maintain proportions of each class in the folds
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [21]:
def save_dataframe(x_train,y_train,x_test,y_test,features,name):
    # Save the final dataset with the 23 features
    X_train = pd.DataFrame(x_train)
    X_train.columns= features
    Y_train = pd.DataFrame(y_train)
    train_frame = [X_train,Y_train]
    train_final = pd.concat(train_frame,axis=1)
    train_final.to_csv('/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/train_data_' + name +'.csv',index = False)
    print("Train dataset Saved")

    # Save test data set as well
    X_test = pd.DataFrame(x_test)
    X_test.columns= features
    Y_test = pd.DataFrame(y_test)
    test_frame = [X_test,Y_test]
    test_final = pd.concat(test_frame,axis=1)
    test_final.to_csv('/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/test_data_'+ name+'.csv',index = False)
    print("Test dataset Saved")

# Read Files and Basic Info

In [22]:
train_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/Train_ULAK.csv'
test_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/Test_ULAK.csv'

train_df = pd.read_csv(train_file_path,skipinitialspace=True)
test_df = pd.read_csv(test_file_path,skipinitialspace=True)

if train_df is not None and test_df is not None:
    print("Data loaded successfully!")
    print("Train Data:")
    print(train_df.head())  # Display the first few rows of the train DataFrame
    print("\nTest Data:")
    print(test_df.head())   # Display the first few rows of the test DataFrame

Data loaded successfully!
Train Data:
   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0                80       68855579                 10                       6   
1                53            196                  2                       2   
2               123            118                  2                       2   
3                80         295657                  7                      10   
4                80       15705351                  7                       5   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                         1038                        11595   
1                           70                          174   
2                           96                           96   
3                         1114                        15841   
4                          407                          452   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                    

In [23]:
# Print the sizes of train and test dataframes
print("Shape of Training Dataset:", train_df.shape)
print("Shape of Testing Dataset:", test_df.shape)

Shape of Training Dataset: (1783356, 79)
Shape of Testing Dataset: (512077, 79)


In [24]:
# number of each label type train df
train_df['Label'].value_counts()

BENIGN                        1432050
DoS Hulk                       145575
PortScan                       100125
DDoS                            80656
DoS GoldenEye                    6484
FTP-Patator                      5000
SSH-Patator                      3714
DoS slowloris                    3651
DoS Slowhttptest                 3464
Bot                              1238
Web Attack � Brute Force          949
Web Attack � XSS                  410
Infiltration                       22
Web Attack � Sql Injection         12
Heartbleed                          6
Name: Label, dtype: int64

In [25]:
# number of each label type test df
test_df['Label'].value_counts()

BENIGN                        411203
DoS Hulk                       41801
PortScan                       28751
DDoS                           23160
DoS GoldenEye                   1861
FTP-Patator                     1436
SSH-Patator                     1067
DoS slowloris                   1048
DoS Slowhttptest                 994
Bot                              355
Web Attack � Brute Force         272
Web Attack � XSS                 117
Infiltration                       6
Web Attack � Sql Injection         4
Heartbleed                         2
Name: Label, dtype: int64

# Data preprocessing

## Drop Records Containing Infitiy Values, are Null and Nan Values

In [26]:
# Replace infinite number with NaN values
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop null and NaN values
print("Records deleted in training dataset:", train_df.isnull().sum().sum())
print("Records deleted in test dataset:", test_df.isnull().sum().sum())

train_df = train_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)

Records deleted in training dataset: 3622
Records deleted in test dataset: 1074


## Label-Encoding to Replace Categorical Values with Numerical

In [27]:
labels = train_df['Label'].values.reshape(-1, 1)
# Initialize the Label Encoder
encoder = LabelEncoder()
# transform fit labels of train and test daframe
train_labels = encoder.fit_transform(labels.reshape(-1,))
test_labels = encoder.transform(test_df['Label'].values.reshape(-1,))
label_order_classes = encoder.classes_
train_df['Label'] = train_labels
test_df['Label'] = test_labels

In [28]:
print(label_order_classes)

['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack � Brute Force' 'Web Attack � Sql Injection'
 'Web Attack � XSS']


In [29]:
# Print the new sizes of train and test dataframes
print("Shape of Training Dataset:", train_df.shape)
print("Shape of Testing Dataset:", test_df.shape)

Shape of Training Dataset: (1781545, 79)
Shape of Testing Dataset: (511540, 79)


## Replace -1 values with the mean of their corresponding column

In [30]:
# Replace -1 with NaN
train_df.replace(-1, np.nan, inplace=True)
test_df.replace(-1, np.nan, inplace=True)
# Calculate the mean of each column
column_means = train_df.mean(skipna=True)

# Fill NaN values with the mean of the corresponding column
train_df.fillna(column_means, inplace=True)
test_df.fillna(column_means,inplace=True)

## Remove the features with zero values

In [31]:
columns_to_drop = ['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk','Fwd Avg Packets/Bulk','Fwd Avg Bulk Rate','Bwd Avg Bytes/Bulk','Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate']

# Drop the specified columns from the DataFrame
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop,inplace=True)

In [32]:
# Print the new sizes of train and test dataframes
print("Shape of Training Dataset:", train_df.shape)
print("Shape of Testing Dataset:", test_df.shape)

Shape of Training Dataset: (1781545, 71)
Shape of Testing Dataset: (511540, 71)


## Split Data

In [38]:
# number of each label type train df
train_df['Label'].value_counts()

0     1430936
4      144976
10     100036
2       80655
3        6484
7        4998
11       3714
6        3651
5        3464
1        1232
12        949
14        410
9          22
13         12
8           6
Name: Label, dtype: int64

In [33]:
X_train = train_df.drop(train_df.columns[-1], axis=1)
y_train = train_df[train_df.columns[-1]]

X_test = test_df.drop(test_df.columns[-1], axis=1)
y_test = test_df[test_df.columns[-1]]

## Save the first version of the dataset with all features

In [18]:
#save without scaling
save_dataframe(X_train,y_train,X_test,y_test,X_train.columns,'70features_nostd')

Train dataset Saved
Test dataset Saved


In [16]:
# scale and save
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)
save_dataframe(x_train,y_train,x_test,y_test,X_train.columns,'70features_mean')

Train dataset Saved
Test dataset Saved


## Oversample categories that are undersampled:


1.   Bot
2.   Heartbleed
3.   Infiltration
4.   Web_Attack_Brute Force
5.   Web_Attack_SqlInjection
6.   Web_Attack_XSS




In [34]:
# Create the SMOTE object with a specific sampling strategy for each class
# Replace the class indices (0, 1, 2) with the indices of the classes you want to over-sample
smote = SMOTE(sampling_strategy={1: 5000,8: 5000, 9: 5000, 11:5000 ,12: 5000, 13: 5000, 14: 5000,}, random_state=42)

#Apply SMOTE to the training data to create synthetic samples
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [37]:
X_train_resampled
#print(y_train_resampled.shape())

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,80,6.885558e+07,10,6,1038,11595,346,0,103.800000,167.133879,...,3,32,998.0,0.0,998,998,68300000.0,0.0,68300000,68300000
1,53,1.960000e+02,2,2,70,174,35,35,35.000000,0.000000,...,1,32,0.0,0.0,0,0,0.0,0.0,0,0
2,123,1.180000e+02,2,2,96,96,48,48,48.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
3,80,2.956570e+05,7,10,1114,15841,1084,0,159.142857,407.829796,...,6,20,0.0,0.0,0,0,0.0,0.0,0,0
4,80,1.570535e+07,7,5,407,452,377,0,58.142857,140.620563,...,6,20,360718.0,0.0,360718,360718,9767208.0,0.0,9767208,9767208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810195,80,5.299261e+06,3,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
1810196,80,5.578751e+06,3,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
1810197,80,5.133065e+06,3,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
1810198,80,5.832812e+06,3,1,0,0,0,0,0.000000,0.000000,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0


In [40]:
y_train_resampled.value_counts()

0     1430936
4      144976
10     100036
2       80655
3        6484
11       5000
14       5000
12       5000
13       5000
1        5000
9        5000
8        5000
7        4998
6        3651
5        3464
Name: Label, dtype: int64

In [41]:
# scale and save
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_resampled)
x_train = scaler.transform(X_train_resampled)
x_test = scaler.transform(X_test)
save_dataframe(x_train,y_train_resampled,x_test,y_test,X_train.columns,'70features_mean_resampled')

Train dataset Saved
Test dataset Saved


Delete initial dataframes to reduce memory consumption

In [ ]:
# Delete dafarames to empty memory
del train_df, test_df

# Manual Recursive Feature Selection using Random Forest feature importance

In [ ]:
# get the indexes of the five folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

Method for Random Forest Feature Importance
*   Uses 5-fold cross validation
*   Keeps features until cumulative importance >= 0.90



In [ ]:
def feature_selection(X,X_test,Y,Y_test,skf,name='default',criterion=0,cumulative=False,save=False):
    # Scale doesn't do anythign after the
    #first iteration since data already scaled
    scaler = preprocessing.StandardScaler()
    scaler.fit(X)
    x_train = scaler.transform(X)
    x_test = scaler.transform(X_test)

    features = X.columns

    feature_importance_scores = np.zeros(X.shape[1])

    for train_index, val_index in skf.split(X,Y):
        X_train, X_val = x_train[train_index],x_train[val_index]
        y_train, y_val = Y[train_index], Y[val_index]

        # Train a Random Forest classifier
        rf = RandomForestClassifier(n_estimators=20, max_depth=None,
                                 bootstrap=False, n_jobs=-1,
                                 random_state=0,verbose=2)
        rf.fit(X_train, y_train)
        y_val_pred = rf.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)
        print("Accuracy:", accuracy)
        # Accumulate feature importance scores
        feature_importance_scores += rf.feature_importances_

    feature_importance_scores /= 5 # divide by 5 to find average F.E.S.

    # Create a DataFrame to store feature importance scores
    feature_importance_df = pd.DataFrame({'feature': features,
                'importance': feature_importance_scores})
    # Sort features by importance scores in descending order
    feature_importance_df = feature_importance_df.sort_values(by='importance',
                                    ascending=False)
    # Keep features based on either a criterion or based on culumative importance
    if cumulative:
        cumulative_importance = 0
        selected_features = []
        indexes = []
        for index, row in feature_importance_df.iterrows():
            if cumulative_importance >= 0.9:
                break
            indexes.append(index)
            selected_features.append(row['feature'])
            cumulative_importance += row['importance']
        return selected_features
    else:
        indexes = feature_importance_df['importance'] > criterion
        selected_features = feature_importance_df[indexes]['feature'].tolist()
        return selected_features
    # save data set if wanted:
    if save:
       save_dataframe(x_train,Y,x_test,Y_test,features,name=name)
    return


## First Iteration


In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',criterion=0,cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


Accuracy: 0.9985826908666354


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20building tree 2 of 20

building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Accuracy: 0.998571464655678


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.8s finished


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 38


## Second Iteration

In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',criterion=0.001,cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.5min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.1s finished


Accuracy: 0.9987286316090809


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.9min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Accuracy: 0.9987033726344269


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.7s finished


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 28


## Third Iteration

In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.5min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


Accuracy: 0.9981897734831284


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20building tree 2 of 20

building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.1min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Accuracy: 0.9982178390105217


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 21


## Fourth Iteration

In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20building tree 2 of 20

building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.3min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


Accuracy: 0.9978726330235834


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

Accuracy: 0.9977800167831854


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.9min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.0s finished


Accuracy: 0.9978782461290622


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 17


## Fifth Iteration

In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.7s finished


Accuracy: 0.997726692281138


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Accuracy: 0.9976200432770432


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.6s finished


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 15


## Sixth Iteration

In [ ]:
selected_features = feature_selection(X_train,X_test,y_train,y_test,skf,name='default',cumulative=True,save=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20building tree 2 of 20

building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.8min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


Accuracy: 0.9977098529647019


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.2min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Accuracy: 0.9976228498297826


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.0s finished


In [ ]:
print("New features reduced to :",len(selected_features))
X_train = X_train.loc[:,selected_features]
X_test = X_test.loc[:,selected_features]

New features reduced to : 13


In [ ]:
model = RandomForestClassifier(n_estimators=20, max_depth=None,
                                 bootstrap=False, n_jobs=-1,
                                 random_state=0,verbose=2)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy on the test set:", accuracy)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.4min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


Model accuracy on the test set: 0.9976697814442663


[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.6s finished


In [ ]:
# print final features
print(selected_features)

['Bwd Packet Length Mean', 'Packet Length Std', 'Total Length of Fwd Packets', 'Bwd Packet Length Std', 'Subflow Fwd Bytes', 'Avg Bwd Segment Size', 'Packet Length Variance', 'Subflow Bwd Bytes', 'Flow IAT Mean', 'Destination Port', 'Init_Win_bytes_forward', 'Total Length of Bwd Packets', 'Fwd Header Length']
